<a href="https://colab.research.google.com/github/vidhan-tiwari/Fake_instagram_account_detection/blob/main/fake_insta_ANN_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from huggingface_hub import login
login()

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import seaborn as sns
%matplotlib inline

In [ ]:
from datasets import load_dataset
import pandas as pd
dataset = load_dataset("nahiar/instagram_bot_detection")

In [ ]:
df = dataset['train'].to_pandas()
df.head()

In [6]:
df = df.sample(frac = 1, random_state = 42).reset_index(drop = True)
X = df.drop("fake",axis = 1)
y = df["fake"]
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X.values, y.values, random_state = 42, test_size = 0.2)

In [7]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)



In [ ]:
from tensorflow.keras.layers import BatchNormalization, Activation
from tensorflow.keras.callbacks import EarlyStopping
model = Sequential()
# layer 1
model.add(Dense(32,input_shape = (X_train.shape[1],)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))
# layer 2
model.add(Dense(64))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))
# output layer
model.add(Dense(1,activation = 'sigmoid'))
model.compile(optimizer = 'adam',loss = 'binary_crossentropy',metrics = ['accuracy'])
model.summary()

# Define the EarlyStopping callback
callback = EarlyStopping(
    monitor='val_loss',
    patience=12,
    restore_best_weights=True
)

# Train

history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=50,
    batch_size=32,
    callbacks=[callback],
    verbose=1
)

In [ ]:
# plot Loss
import matplotlib.pyplot as plt
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss Evolution')
plt.legend()

# plot Accuracy
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy Evolution')
plt.legend()
plt.show()

In [12]:
from sklearn.metrics import classification_report

y_pred_proba = model.predict(X_test)
y_pred_binary = (y_pred_proba > 0.5).astype(int)

print(classification_report(y_test, y_pred_binary))

32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step
              precision    recall  f1-score   support

           0       0.94      0.99      0.97       482
           1       0.99      0.94      0.97       518

    accuracy                           0.97      1000
   macro avg       0.97      0.97      0.97      1000
weighted avg       0.97      0.97      0.97      1000



In [ ]:
model.save('instagram_fake_detector.h5')

import joblib
joblib.dump(scaler, 'scaler.pkl')

In [ ]:
all_records = {
    "Person 1": np.array([1, 0.25,  7, 0, 0, 19, 0, 1, 0, 277, 220]), # person_1
    "Person 2": np.array([1, 0.17, 22, 0, 0, 37, 0, 1, 7, 343, 308]), # person_2
    "Person 3": np.array([1, 0.16, 13, 0, 0, 0,  0, 1, 0, 302, 736]), # person_3
    "Person 4": np.array([1, 0.11, 17, 0, 0, 0,  0, 1, 0, 127, 247]), # person_4
    "Random 1": np.array([0, 0.05,  6, 0.15, 0, 32, 1, 0, 126, 6000, 7500]) # Suspicious
}

names = list(all_records.keys())
custom_data = np.array(list(all_records.values()))

# data scaling using standard scaler
custom_data_scaled = scaler.transform(custom_data)

# Data prediction
predictions = model.predict(custom_data_scaled)


print("-" * 50)
print(f"{'Name':<15} | {'Probability (Fake)':<20} | {'Result'}")
print("-" * 50)

for i, name in enumerate(names):
    prob_fake = predictions[i][0]

    result = "FAKE ACCOUNT" if prob_fake > 0.5 else "Real Account"

    print(f"{name:<15} | {prob_fake:.8f}           | {result}")

print("-" * 50)